In [29]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

# Base URL 
base_url = "https://atl-01.statsplus.net/skylinebaseball/reports/news/html/coaches/human_manager_{}.html"

# Init managing histories
all_managing_histories = []

# Loop through manager IDs from 1 to 95 (max I could find for most recent GMs hopefully shit didn't get deleted)
for manager_id in range(1, 96):  
    url = base_url.format(manager_id)
    
    try:
        # Fetch webpage
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Get Name
        gm_name = soup.find('div', class_='reptitle').get_text(strip=True).replace("General Manager ", "")
        
        # Find managing history table
        managing_history_table = soup.find('table', class_='data sortable')
        if not managing_history_table:
            continue  # Skip if no table is found
        
        # Loop thru table
        for row in managing_history_table.find_all('tr')[1:]:
            columns = row.find_all('td')
            if columns:
                year = columns[0].get_text(strip=True)
                team = columns[2].get_text(strip=True)
                
                 # Extract the ID from the href attribute
                team_link = columns[2].find('a')['href']  # Extract the href value
                team_id_match = re.search(r'team_(\d+)', team_link)  # Extract numeric ID
                team_id = team_id_match.group(1) if team_id_match else None
                
                wins = columns[4].get_text(strip=True)
                losses = columns[5].get_text(strip=True)
                win_pct = columns[6].get_text(strip=True)
                finish = columns[7].get_text(strip=True)
                all_managing_histories.append([gm_name, year, team, team_id, wins, losses, win_pct, finish])
    
    except Exception as e:
        print(f"Error processing URL {url}: {e}")

# Convert to df
columns = ["GM", "Year", "Team", "Team_ID", "W", "L", "WPct", "Finish"]
df = pd.DataFrame(all_managing_histories, columns=columns)


print(df) 
df.to_csv("managing_history.csv", index=False)  

Error processing URL https://atl-01.statsplus.net/skylinebaseball/reports/news/html/coaches/human_manager_15.html: 404 Client Error: Not Found for url: https://atl-01.statsplus.net/skylinebaseball/reports/news/html/coaches/human_manager_15.html
Error processing URL https://atl-01.statsplus.net/skylinebaseball/reports/news/html/coaches/human_manager_95.html: 404 Client Error: Not Found for url: https://atl-01.statsplus.net/skylinebaseball/reports/news/html/coaches/human_manager_95.html
              GM  Year        Team Team_ID    W   L  WPct Finish
0     Joe Peters  2000  Pittsburgh      52   57  99  .365    8th
1     Joe Peters  2001  Pittsburgh      52   74  82  .474    5th
2     Joe Peters  2002  Pittsburgh      52   91  65  .583    1st
3     Joe Peters  2003  Pittsburgh      52  105  51  .673    1st
4     Joe Peters  2004  Pittsburgh      52   94  62  .603    1st
...          ...   ...         ...     ...  ...  ..   ...    ...
2063   Ettin One  2054  Pittsburgh      52    0   0  .0